# 1. Overview & Objectives  
Brief description of what this notebook is responsible for.

- What type of models are implemented here
    - LSTM
    - NBEATS
    - NHITS
    - GRU
    - RNN

- Expected outputs (e.g., CSV with metrics, plots, forecasts)

All models should be implemented in such a way that in the visualizations.ipynb it will be possible to call a function for the model using the parameters from the `best_param.csv` file for easy of plotting on the time series. Should be possible for test & val

All the best models should be saved in the `best_param.csv` file


# 2. Imports & Setup

In [22]:
# Importing the helper notebooks

## Enable imports from .ipynb files
import import_ipynb
import sys
sys.path.append("code")

## Importing the helper notebooks as modules
from splitting import split_time_series
from metrics import evaluate_and_save, load_best_models
from save_as_csv import save_forecast_parquet

# Notebook specific imports
# Core libraries
import numpy as np
import pandas as pd
from pathlib import Path

# Nixtla
from neuralforecast import NeuralForecast
from neuralforecast.models import LSTM, NBEATS, NHITS, GRU, RNN
from neuralforecast.losses.pytorch import MAE

# Torch (for reproducibility)
import torch
torch.manual_seed(42)

# 3. Load Data & Train/Val/Test Split
use `split_time_series()`

In [2]:
splits = split_time_series()

train_df = splits["train"]
val_df   = splits["val"]
test_df  = splits["test"]

TARGET_COL = "tavg"

In [3]:
def to_nixtla_format(df, uid="series_1"):
    out = df.rename(columns={"time": "ds", "tavg": "y"}).copy()
    out["unique_id"] = uid
    return out[["unique_id", "ds", "y"]]

train_nf = to_nixtla_format(train_df)
val_nf   = to_nixtla_format(val_df)
test_nf  = to_nixtla_format(test_df)

# 4. Model Definition  
Clearly specify:  
- Model names  
- Hyperparameters  

In [9]:
HORIZON = len(val_nf)
INPUT_SIZE = 365
MAX_STEPS = 500

def make_name(base, **params):
    return "_".join([base] + [f"{k}{v}" for k, v in params.items()])

In [ ]:
models = [
    (
        make_name("LSTM", h=HORIZON, in_=INPUT_SIZE, enc="64x2"),
        LSTM(
            h=HORIZON,
            input_size=INPUT_SIZE,
            encoder_hidden_size=64,
            encoder_n_layers=2,
            decoder_hidden_size=64,
            decoder_layers=2,
            loss=MAE(),
            max_steps=MAX_STEPS
        )
    ),

    (
        make_name("NBEATS", h=HORIZON, in_=INPUT_SIZE),
        NBEATS(
            h=HORIZON,
            input_size=INPUT_SIZE,
            loss=MAE(),
            max_steps=MAX_STEPS
        )
    ),

    (
        make_name("NHITS", h=HORIZON, in_=INPUT_SIZE),
        NHITS(
            h=HORIZON,
            input_size=INPUT_SIZE,
            loss=MAE(),
            max_steps=200
        )
    ),
    (
        make_name("GRU", h=HORIZON, in_=INPUT_SIZE, enc="64x2"),
        GRU(
            h=HORIZON,
            input_size=INPUT_SIZE,
            encoder_hidden_size=64,
            encoder_n_layers=2,
            decoder_hidden_size=64,
            decoder_layers=2,
            loss=MAE(),
            max_steps=MAX_STEPS
        )
    ),
    (
        make_name("RNN", h=HORIZON, in_=INPUT_SIZE, enc="64x2"),
        RNN(
            h=HORIZON,
            input_size=INPUT_SIZE,
            encoder_hidden_size=64,
            encoder_n_layers=2,
            decoder_hidden_size=64,
            decoder_layers=2,
            loss=MAE(),
            max_steps=MAX_STEPS
        )
    ),
]


Seed set to 1
Seed set to 1
Seed set to 1
Seed set to 1
Seed set to 1


# 5. Training  
For each model:  
- Fit on training data  
- (For ML & Neural) prepare features / loaders / windows

## 5.1 For validation

In [11]:
H_VAL = len(val_nf)

models_val = [
    (
        make_name("LSTM", h=H_VAL, in_=INPUT_SIZE, enc="64x2"),
        LSTM(
            h=H_VAL,
            input_size=INPUT_SIZE,
            encoder_hidden_size=64,
            encoder_n_layers=2,
            decoder_hidden_size=64,
            decoder_layers=2,
            loss=MAE(),
            max_steps=MAX_STEPS
        )
    ),

    (
        make_name("NBEATS", h=H_VAL, in_=INPUT_SIZE),
        NBEATS(
            h=H_VAL,
            input_size=INPUT_SIZE,
            loss=MAE(),
            max_steps=MAX_STEPS
        )
    ),

    (
        make_name("NHITS", h=H_VAL, in_=INPUT_SIZE),
        NHITS(
            h=H_VAL,
            input_size=INPUT_SIZE,
            loss=MAE(),
            max_steps=200
        )
    ),
    (
        make_name("GRU", h=H_VAL, in_=INPUT_SIZE, enc="64x2"),
        GRU(
            h=H_VAL,
            input_size=INPUT_SIZE,
            encoder_hidden_size=64,
            encoder_n_layers=2,
            decoder_hidden_size=64,
            decoder_layers=2,
            loss=MAE(),
            max_steps=MAX_STEPS
        )
    ),
    (
        make_name("RNN", h=H_VAL, in_=INPUT_SIZE, enc="64x2"),
        RNN(
            h=H_VAL,
            input_size=INPUT_SIZE,
            encoder_hidden_size=64,
            encoder_n_layers=2,
            decoder_hidden_size=64,
            decoder_layers=2,
            loss=MAE(),
            max_steps=MAX_STEPS
        )
    ),
]


nf_val = NeuralForecast(
    models=[m for _, m in models_val],
    freq="D"
)

Seed set to 1
Seed set to 1
Seed set to 1
Seed set to 1
Seed set to 1


## 5.2 For Test

In [12]:
H_TEST = len(test_nf)

models_test = [
    (
        make_name("LSTM", h=H_TEST, in_=INPUT_SIZE, enc="64x2"),
        LSTM(
            h=H_TEST,
            input_size=INPUT_SIZE,
            encoder_hidden_size=64,
            encoder_n_layers=2,
            decoder_hidden_size=64,
            decoder_layers=2,
            loss=MAE(),
            max_steps=MAX_STEPS
        )
    ),

    (
        make_name("NBEATS", h=H_TEST, in_=INPUT_SIZE),
        NBEATS(
            h=H_TEST,
            input_size=INPUT_SIZE,
            loss=MAE(),
            max_steps=MAX_STEPS
        )
    ),

    (
        make_name("NHITS", h=H_TEST, in_=INPUT_SIZE),
        NHITS(
            h=H_TEST,
            input_size=INPUT_SIZE,
            loss=MAE(),
            max_steps=200
        )
    ),
    (
        make_name("GRU", h=H_TEST, in_=INPUT_SIZE, enc="64x2"),
        GRU(
            h=H_TEST,
            input_size=INPUT_SIZE,
            encoder_hidden_size=64,
            encoder_n_layers=2,
            decoder_hidden_size=64,
            decoder_layers=2,
            loss=MAE(),
            max_steps=MAX_STEPS
        )
    ),
    (
        make_name("RNN", h=H_TEST, in_=INPUT_SIZE, enc="64x2"),
        RNN(
            h=H_TEST,
            input_size=INPUT_SIZE,
            encoder_hidden_size=64,
            encoder_n_layers=2,
            decoder_hidden_size=64,
            decoder_layers=2,
            loss=MAE(),
            max_steps=MAX_STEPS
        )
    ),
]


nf_test = NeuralForecast(
    models=[m for _, m in models_test],
    freq="D"
)

Seed set to 1
Seed set to 1
Seed set to 1
Seed set to 1
Seed set to 1


# 6. Forecasting  
- Produce forecasts for validation and test horizons

## 6.1 Validation Forecast

In [13]:
nf_val.fit(train_nf)
val_forecasts = nf_val.predict()

GPU available: False, used: False
TPU available: False, using: 0 TPU cores

  | Name              | Type          | Params | Mode 
------------------------------------------------------------
0 | loss              | MAE           | 0      | train
1 | padder_train      | ConstantPad1d | 0      | train
2 | scaler            | TemporalNorm  | 0      | train
3 | hist_encoder      | LSTM          | 50.4 K | train
4 | mlp_decoder       | MLP           | 4.2 K  | train
5 | upsample_sequence | Linear        | 668 K  | train
------------------------------------------------------------
723 K     Trainable params
0         Non-trainable params
723 K     Total params
2.893     Total estimated model params size (MB)
11        Modules in train mode
0         Modules in eval mode


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

...

Predicting: |          | 0/? [00:00<?, ?it/s]

GPU available: False, used: False
TPU available: False, using: 0 TPU cores


Predicting: |          | 0/? [00:00<?, ?it/s]

GPU available: False, used: False
TPU available: False, using: 0 TPU cores


Predicting: |          | 0/? [00:00<?, ?it/s]

In [14]:
val_forecasts = val_forecasts.rename(
    columns={
        old: name
        for (name, _), old in zip(models_val, val_forecasts.columns[2:])
    }
)


## 6.2 Test Forecast

In [15]:
train_val_nf = pd.concat([train_nf, val_nf])

nf_test.fit(train_val_nf)
test_forecasts = nf_test.predict()

GPU available: False, used: False
TPU available: False, using: 0 TPU cores

  | Name              | Type          | Params | Mode 
------------------------------------------------------------
0 | loss              | MAE           | 0      | train
1 | padder_train      | ConstantPad1d | 0      | train
2 | scaler            | TemporalNorm  | 0      | train
3 | hist_encoder      | LSTM          | 50.4 K | train
4 | mlp_decoder       | MLP           | 4.2 K  | train
5 | upsample_sequence | Linear        | 655 K  | train
------------------------------------------------------------
710 K     Trainable params
0         Non-trainable params
710 K     Total params
2.842     Total estimated model params size (MB)
11        Modules in train mode
0         Modules in eval mode


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

...

Predicting: |          | 0/? [00:00<?, ?it/s]

GPU available: False, used: False
TPU available: False, using: 0 TPU cores


Predicting: |          | 0/? [00:00<?, ?it/s]

GPU available: False, used: False
TPU available: False, using: 0 TPU cores


Predicting: |          | 0/? [00:00<?, ?it/s]

In [16]:
test_forecasts = test_forecasts.rename(
    columns={
        old: name
        for (name, _), old in zip(models_test, test_forecasts.columns[2:])
    }
)

# 7. Evaluation (Using Shared Metrics Function)  
- Apply `evaluate_and_save()` to each model  
- Save results as CSV into `data/models/`  
- Display sorted results table  

In [17]:
OUT_FILE = "neural_models_results.csv"
BEST_PARAM_FILE = "best_param.csv"

results = []
best_params = []

def evaluate_split(y_true, forecasts, split_name):
    for model_name in forecasts.columns:
        if model_name in ["unique_id", "ds"]:
            continue

        metrics = evaluate_and_save(
            y_true=y_true,
            y_pred=forecasts[model_name].values,
            model_name=model_name,
            impl_name="neural",
            split_name=split_name,
            out_filename=OUT_FILE
        )

        results.append(metrics)

        best_params.append({
            "Model": model_name,
            "Impl": "neural"
        })

In [18]:
# Validation
evaluate_split(
    y_true=val_nf["y"].values,
    forecasts=val_forecasts,
    split_name="val"
)

In [19]:
# Test
evaluate_split(
    y_true=test_nf["y"].values,
    forecasts=test_forecasts,
    split_name="test"
)

In [20]:
load_best_models("neural_models_results.csv", split="test")

,Model,Impl,Split,MAE,RMSE,MAPE,OPE,R2
0,GRU_h1792_in_365_enc64x2,neural,test,3.503292,4.444511,7.262492e+09,0.110348,0.812318
1,LSTM_h1792_in_365_enc64x2,neural,test,3.514066,4.464777,7.235003e+09,0.117615,0.810603
2,NHITS_h1792_in_365,neural,test,3.546532,4.532260,6.297819e+09,0.137575,0.804834
3,NBEATS_h1792_in_365,neural,test,3.552858,4.539296,7.437694e+09,0.159560,0.804228
4,RNN_h1792_in_365_enc64x2,neural,test,3.600080,4.572447,7.046777e+09,0.162688,0.801358


In [21]:
load_best_models("neural_models_results.csv", split="val")

,Model,Impl,Split,MAE,RMSE,MAPE,OPE,R2
0,NHITS_h1827_in_365,neural,val,3.583653,4.554383,3.111514e+09,0.054766,0.817724
1,LSTM_h1827_in_365_enc64x2,neural,val,3.586652,4.548621,2.721471e+09,0.007795,0.818185
2,NBEATS_h1827_in_365,neural,val,3.596953,4.573221,3.377013e+09,0.005202,0.816213
3,GRU_h1827_in_365_enc64x2,neural,val,3.603196,4.566222,3.246916e+09,0.020163,0.816776
4,RNN_h1827_in_365_enc64x2,neural,val,3.604439,4.569031,2.934331e+09,0.004720,0.816550


# 8. Conclusions  
Short wrap-up:  
- Which model family performed best here?  
- Any issues or instability?  
- Notes for integration in the final report  

Best performing model was **GRU**

With parameters:
```
h=HORIZON,
input_size=INPUT_SIZE,
encoder_hidden_size=64,
encoder_n_layers=2,
decoder_hidden_size=64,
decoder_layers=2,
loss=MAE(),
max_steps=MAX_STEPS
```

MAE: 
of 3.5 degrees

# Saving the best pred of the best model

using save_forecast_parquet

## Retraining and predicting into the future

In [56]:
H_FUTURE = 62 # because we are gonna save forecasts from Dec 1, 2025 to January 31, 2026

In [57]:
future_models = [
    (
        make_name("GRU_future", h=H_FUTURE, in_=INPUT_SIZE, enc="64x2"),
        GRU(
            h=H_FUTURE,
            input_size=INPUT_SIZE,
            encoder_hidden_size=64,
            encoder_n_layers=2,
            decoder_hidden_size=64,
            decoder_layers=2,
            loss=MAE(),
            max_steps=MAX_STEPS
        )
    )
]


Seed set to 1


In [58]:
full_nf = pd.concat([train_nf, val_nf, test_nf])

nf_future = NeuralForecast(
    models=[m for _, m in future_models],
    freq="D"
)

nf_future.fit(full_nf)

GPU available: False, used: False
TPU available: False, using: 0 TPU cores



  | Name         | Type          | Params | Mode 
-------------------------------------------------------
0 | loss         | MAE           | 0      | train
1 | padder_train | ConstantPad1d | 0      | train
2 | scaler       | TemporalNorm  | 0      | train
3 | hist_encoder | GRU           | 37.8 K | train
4 | mlp_decoder  | MLP           | 4.2 K  | train
-------------------------------------------------------
42.0 K    Trainable params
0         Non-trainable params
42.0 K    Total params
0.168     Total estimated model params size (MB)
10        Modules in train mode
0         Modules in eval mode


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

`Trainer.fit` stopped: `max_steps=500` reached.


In [59]:
future_forecasts = nf_future.predict()

GPU available: False, used: False
TPU available: False, using: 0 TPU cores


Predicting: |          | 0/? [00:00<?, ?it/s]

In [ ]:
future_forecasts

,unique_id,ds,GRU
0,series_1,2025-11-28,-0.120616
1,series_1,2025-11-29,-0.248888
2,series_1,2025-11-30,-0.420237
3,series_1,2025-12-01,-0.561464
4,series_1,2025-12-02,-0.729156
...,...,...,...
57,series_1,2026-01-24,-7.441684
58,series_1,2026-01-25,-7.439586
59,series_1,2026-01-26,-7.409603
60,series_1,2026-01-27,-7.400577


## Saving ypred in a parquet file

In [ ]:
print(future_forecasts.columns.tolist())

['unique_id', 'ds', 'GRU']


In [66]:
# future_forecasts comes from nf_future.predict()
best_pred = future_forecasts[["ds", "GRU"]].copy()
best_pred = best_pred.rename(columns={"GRU": "y_pred"})

In [67]:
# restrict to the period to Dec 2025 to Jan 2026
best_pred = best_pred[
    (best_pred["ds"] >= "2025-12-01") &
    (best_pred["ds"] <= "2026-01-31")
]

In [68]:
best_pred

,ds,y_pred
3,2025-12-01,-0.561464
4,2025-12-02,-0.729156
5,2025-12-03,-0.940488
6,2025-12-04,-0.993537
7,2025-12-05,-0.931486
8,2025-12-06,-0.849805
9,2025-12-07,-0.846331
10,2025-12-08,-1.051448
11,2025-12-09,-1.377437
12,2025-12-10,-1.737280


In [70]:
save_forecast_parquet(
    dates = best_pred["ds"],
    y_pred = best_pred["y_pred"],
    filename="best_neural_forecast_dec_jan.parquet"
)


WindowsPath('../data/models/best_neural_forecast_dec_jan.parquet')

In [71]:
# read in the parquet file to verify
df_check = pd.read_parquet("../data/models/best_neural_forecast_dec_jan.parquet")

In [72]:
df_check

,date,y_pred,y_actual
0,2025-12-01,-0.561464,NaN
1,2025-12-02,-0.729156,NaN
2,2025-12-03,-0.940488,NaN
3,2025-12-04,-0.993537,NaN
4,2025-12-05,-0.931486,NaN
5,2025-12-06,-0.849805,NaN
6,2025-12-07,-0.846331,NaN
7,2025-12-08,-1.051448,NaN
8,2025-12-09,-1.377437,NaN
9,2025-12-10,-1.737280,NaN


In [73]:
df_check.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 59 entries, 0 to 58
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype         
---  ------    --------------  -----         
 0   date      59 non-null     datetime64[ns]
 1   y_pred    59 non-null     float64       
 2   y_actual  0 non-null      float64       
dtypes: datetime64[ns](1), float64(2)
memory usage: 1.5 KB


In [74]:
df_check.describe()

,date,y_pred,y_actual
count,59,59.000000,0.0
mean,2025-12-30 00:00:00,-4.223677,NaN
min,2025-12-01 00:00:00,-7.441684,NaN
25%,2025-12-15 12:00:00,-6.428927,NaN
50%,2025-12-30 00:00:00,-4.077947,NaN
75%,2026-01-13 12:00:00,-2.339280,NaN
max,2026-01-28 00:00:00,-0.561464,NaN
std,NaN,2.282342,NaN
